# 시계열 토픽 모델링(Dynamic Topic Modeling)

### 패키지 설치

In [3]:
# 나눔고딕 설치
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum.
(Reading database ... 145480 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [5]:
# dtm_binaries 다운로드
!mkdir /dtm_binaries
!wget -P ./dtm_binaries https://github.com/ByungjunKim/DDMKL/raw/main/dtm_binaries/dtm-linux64

--2021-01-04 08:11:06--  https://github.com/ByungjunKim/DDMKL/raw/main/dtm_binaries/dtm-linux64
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ByungjunKim/DDMKL/main/dtm_binaries/dtm-linux64 [following]
--2021-01-04 08:11:06--  https://raw.githubusercontent.com/ByungjunKim/DDMKL/main/dtm_binaries/dtm-linux64
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3601176 (3.4M) [application/octet-stream]
Saving to: ‘./dtm_binaries/dtm-linux64’

dtm-linux64         100%[===================>]   3.43M  --.-KB/s    in 0.1s    

2021-01-04 08:11:06 (27.3 MB/s) - ‘./dtm_binaries/dtm-linux64’ saved [3601176/360117

In [ ]:
!wget https://github.com/ByungjunKim/DDMKL/raw/main/phd.pickle

In [46]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm
tqdm.pandas()
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.callbacks import PerplexityMetric
from gensim.models.wrappers import DtmModel
# https://github.com/magsilva/dtm/tree/master/bin
# path_to_dtm_binary = "./dtm_binaries/dtm-win64.exe" # 윈도우
path_to_dtm_binary = "./dtm_binaries/dtm-linux64"
!chmod -R 777 ./dtm_binaries/dtm-linux64
from gensim.matutils import corpus2csc
from scipy.spatial.distance import cosine
from scipy.stats import linregress

# Plotting tools
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

# Enable logging for gensim - optional
import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# networkx
import networkx as nx

# DTM plot 모듈 로드(dtm_plot.py)
import dtm_plot as dp

### 서지정보 데이터 로드

In [8]:
!wget https://github.com/ByungjunKim/DDMKL/raw/main/phd.pickle

--2021-01-04 08:14:24--  https://github.com/ByungjunKim/DDMKL/raw/main/phd.pickle
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ByungjunKim/DDMKL/main/phd.pickle [following]
--2021-01-04 08:14:24--  https://raw.githubusercontent.com/ByungjunKim/DDMKL/main/phd.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19321951 (18M) [application/octet-stream]
Saving to: ‘phd.pickle’

phd.pickle          100%[===================>]  18.43M  70.6MB/s    in 0.3s    

2021-01-04 08:14:25 (70.6 MB/s) - ‘phd.pickle’ saved [19321951/19321951]



In [9]:
with open('phd.pickle', 'rb') as f:
    phd = pickle.load(f)

In [10]:
phd

,Index,대주제,장르,중주제1,중주제2,대상작가,시대별,서명,기타서명,저자,발행사항,대학,형태사항,일반주기명,학위논문사항,DDC,KDC,발행국,출판년,주제어,소장기관,UCI식별코드,국문 초록 (Abstract),다국어 초록 (Multilingual Abstract),목차,contents,tokens
0,1,작가론,시,시인론,NaN,서정주,통사,"徐廷柱 詩 硏究 = A study on the Poetry of Suh, Jung-Ju",NaN,연은순,"청주 : 청주대학교, 2000",청주대학교,174 p. ; 26 cm.,NaN,학위논문(박사)-- 청주대학교 대학원 : 국어국문학과 국문학전공 2000,NaN,811.6,서울,2000,,"강원대학교 도서관 (242002),국립중앙도서관 (011001),단국대학교 퇴계기념...",NaN,,"This dissertation aims at investigating Suh, J...",목차\r\nI. 서론 = 1\r\n 1. 문제제기 = 1\r\n 2. 연구사 검토 ...,서정주 시 연구 서정주 시 연구 ...,"[서정주, 시, 서정주, 시, 시어, 구사, 비문법성, 시성, 표출, 모순어법, 미..."
1,2,주제론,소설,소설주제론,NaN,NaN,1950,전후소설과 이데올로기의 상관성 연구 = (A) Study of Correlation...,NaN,이봉일,"서울 : 경희대학교 대학원, 2000",경희대학교,155p. : 삽도 ; 26cm.,NaN,학위논문(박사)-- 경희대학원 대학원: 국어국문학과 2000,NaN,813.6 4,서울,2000,전후소설 이데올로기 상관성,"경희대학교 중앙도서관 (211006),국립중앙도서관 (011001)",NaN,본논문은 '50년대 전후소설과 이데올로기의 상관성을 이해하는 데 목적이 있다. 그동...,The aim of this dissertation is to understand ...,Ⅰ. 서론 = 1\r\n 1. 문제제기 = 1\r\n 2. 연구사 검토 = 5\r\...,전후소설과 이데올로기의 상관성 연구 전후소설과 ...,"[전후, 소설, 이데올로기, 상관성, 전후, 소설, 이데올로기, 상관성, 전쟁, 이..."
2,3,주제론,비평,소설주제론,NaN,NaN,개화기,韓國 近代初期 文學論과 小說化 樣相 硏究,NaN,김윤재,"서울 : 韓國外國語大學校 大學院, 2000",한국외국어대학교,"ii, 165 p. ; 26 cm.",참고문헌 : p.158-163.,학위논문(박사)-- 韓國外國語大學校大學院 : 國語國文學 2000,895.1309 16,NaN,서울,2000,,"국립중앙도서관 (011001),한국외국어대학교 서울캠퍼스 도서관 (211059)",NaN,,NaN,"\n=0,1,3\n차례=i,4,2\nI. 서론=1,6,1\n1. 연구목적=1,6,2...",한국 근대초기 문학론과 소설화 양상 연구 한국 근대초기 문학론과 소설화 양상 연구 ...,"[근대, 초기, 문학론, 소설화, 근대, 초기, 문학론, 소설화, 차례, 현황, 계..."
3,4,비교문학,비평,비교문학,시인론,"조지훈,크리슈나무르티",통사,조지훈과 크리슈나무르티의 선적(禪的) 동질성 연구,NaN,김기호,"서울 : 한국외국어대학교 대학원, 2000",한국외국어대학교,163 p. ; 26 cm.,참고문헌 : p.154-159.,학위논문(박사)-- 한국외국어대학교대학원 : 국어국문 2000,895.1109 16,NaN,서울,2000,,"국립중앙도서관 (011001),한국외국어대학교 서울캠퍼스 도서관 (211059)",NaN,,NaN,"\n=0,1,3\n목차=i,4,1\nI. 서론=1,5,1\n1. 문제의 제기=1,5...",조지훈과 크리슈나무르티의 선적 선적 동질성 연구 조지훈과 크리슈나무르티의 선적 선...,"[조지훈, 크리슈나무르티, 선, 선, 동질성, 조지훈, 크리슈나무르티, 선, 선, ..."
4,5,주제론,소설,여성문학_젠더(해방 이후),소설주제론,NaN,1950,1950년대 한국소설의 여성인물 연구 : 현실수용양상을 중심으로 = (A)study...,NaN,송경란,"서울: 淑明女子大學校, 2000",숙명여자대학교,"iv, 158p.; 26cm",NaN,학위논문(박사) -- 숙명여자대학교 대학원: 국어국문학과 현대문학전공 2000,895.734 21,813.6 4,서울,2000,1950년대 소설 여성인물,"국립중앙도서관 (011001),숙명여자대학교 도서관 (211043),한국학중앙연구원...",NaN,이 논문은 1950년대 소설에서 전쟁과 전후상황에 의한 여성인물의 현실수용양상을 살...,"In this study, I tried to explain the influenc...",Ⅰ. 서론 ········································...,1950년대 한국소설의 여성인물 연구 현실수용양상을 중심으로 ...,"[1950년대, 소설, 여성, 인물, 현실, 수용, 1950, 1950년대, 소설,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,1524,작가론,소설,소설작가론,NaN,서정인,해방이후,서정인 소설의 변모 양상 연구 = A Study on Changing Aspects...,NaN,박희현,"서울 : 서울시립대학교 인문대학원, 2019",서울시립대학교,"ⅱ, 258 p. ; 26 cm",지도교수: 이동하,학위논문(박사)-- 서울시립대학교 인문대학원 : 국어국문학과 현대문학 2019. 8,NaN,NaN,서울,2019,미메시스 디에게시스 말의 중첩 언어적 다양성 상대주의적 세계관 인유(引喩) 판소리 ...,서울시립대학교 도서관 (211035),I804:11035-000000030867,본고는 서정인 소설 세계의 변모 양상을 작품을 통해 구체적으로 살펴보는 것을 연구의...,The purpose of this study is to examine aspect...,제1장. 서론 1&#xD;\n 제1절. 문제 제기 및 연구 목적 1&#xD;...,서정인 소설의 변모 양상 연구 서정인 소설의 변...,"[서정인, 소설, 변모, 서정인, 소설, 변모, 서술, 순서, 시대, 현실, 충실,..."
1524,1525,문화,기타_복합장르,여성문학_젠더(해방 이후),문학문화,NaN,1960~1970,'여류문학전집'(1967-1979)과 여성문학의 젠더 정치,NaN,김은석,"서울: 동국대학교, 2019",동국대학교,"iii, 189 p.; 26 cm.","동국대학교 논문은 저작권에 의해 보호받습니다.,지도교수: 김춘식,서지적 각주와 참고...",학위논문(박사)-- 동국대학교 대학원: 국어국문학과 2019. 2,NaN,NaN,서울,2019,여류문학전집 문학 장 한국여류문학인회 문학 장 정전,동국대학교 중앙도서관 (211020),I804:11020-000000079013,본 논문은 ‘여류문학전집’(1967~1979)을 중심으로 1960~70년대 여류문학...,The Women’s Literary Collection by Women Write...,Ⅰ. 서 론 1&#xD;\n 1. 연구 목적 및 연구사 검토 1&#xD;\n 2...,여류문학전집 1967 1979 과 여성문학의 젠더 정치 여류문학전집 1967 1...,"[여류, 전집, 여성, 젠더, 정치, 여류, 전집, 여성, 젠더, 정치, 1960년..."
1525,1526,작가론,비평,비평사,NaN,이어령,해방이후,이어령 문학비평 연구 = A Study of Lee O-young′s Literar...,NaN,홍래성,"서울 : 서울시립대학교 일반대학원, 2019",서울시립대학교,"ⅰ, 268 p. ; 26 cm",지도교수: 이동하,학위논문(박사)-- 서울시립대학교 일반대학원 : 국어국문학과 2019. 8,NaN,NaN,서울,2019,이어령 1950년대 전후세대 이상론 저항의 문학 퍼포머티브 전통론 김동리 사르트르 ...,서울시립대학교 도서관 (211035),I804:11035-000000030919,본 논문은 전후의 시기에 펼쳐진 이어령의 문학비평을 대상으로 그 

In [11]:
phd['tokens']

0       [서정주, 시, 서정주, 시, 시어, 구사, 비문법성, 시성, 표출, 모순어법, 미...
1       [전후, 소설, 이데올로기, 상관성, 전후, 소설, 이데올로기, 상관성, 전쟁, 이...
2       [근대, 초기, 문학론, 소설화, 근대, 초기, 문학론, 소설화, 차례, 현황, 계...
3       [조지훈, 크리슈나무르티, 선, 선, 동질성, 조지훈, 크리슈나무르티, 선, 선, ...
4       [1950년대, 소설, 여성, 인물, 현실, 수용, 1950, 1950년대, 소설,...
                              ...                        
1523    [서정인, 소설, 변모, 서정인, 소설, 변모, 서술, 순서, 시대, 현실, 충실,...
1524    [여류, 전집, 여성, 젠더, 정치, 여류, 전집, 여성, 젠더, 정치, 1960년...
1525    [어령, 비평, 어령, 비평, 필요성, 시각, 순서, 비평, 사유, 배경, 비평, ...
1526    [이현화, 희곡, 대화, 현실, 인식, 이현화, 희곡, 대화, 현실, 인식, 국문,...
1527    [중국, 일본, 근대, 유미주의, 비교, 김동인, 욱달부, 곡기윤이, 중국, 일본,...
Name: tokens, Length: 1528, dtype: object

In [20]:
# 토픽 모델링에 활용할 시계열 메타변수(time_slice) 컬럼 만들기
phd['t'] = phd['출판년']-2000
phd['t']

0        0
1        0
2        0
3        0
4        0
        ..
1523    19
1524    19
1525    19
1526    19
1527    19
Name: t, Length: 1528, dtype: int64

### 모델링

In [25]:
def prepare_DTM(df,min_count,threshold):
    time_slice = df['tokens'].groupby(df['t']).size()
    data_words = df['tokens'].to_list()
    bigram = gensim.models.Phrases(data_words, min_count=min_count,threshold=threshold)
    bigram_mod =  gensim.models.phrases.Phraser(bigram)
    data_words_bigrams = [bigram_mod[doc] for doc in data_words]
    id2word = corpora.Dictionary(data_words_bigrams)
    corpus = [id2word.doc2bow(text) for text in data_words_bigrams]
    del data_words
    return time_slice, id2word, corpus

In [26]:
# 최소 100회이상 등장한 단어만 토픽모델링에 활용
time_slice, id2word, corpus = prepare_DTM(phd,100,50)

2021-01-04 08:27:37,374 : INFO : collecting all words and their counts
2021-01-04 08:27:37,376 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-01-04 08:27:37,691 : INFO : collected 123945 word types from a corpus of 158072 words (unigram + bigrams) and 1528 sentences
2021-01-04 08:27:37,691 : INFO : using 123945 counts as vocab in Phrases<0 vocab, min_count=100, threshold=50, max_vocab_size=40000000>
2021-01-04 08:27:37,692 : INFO : source_vocab length 123945
2021-01-04 08:27:39,086 : INFO : Phraser built with 0 phrasegrams
2021-01-04 08:27:39,594 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-01-04 08:27:39,750 : INFO : built Dictionary(14545 unique tokens: ['구사', '능청떨기', '대지', '모순어법', '미학']...) from 1528 documents (total 158072 corpus positions)


In [44]:
DTM_8 = DtmModel(path_to_dtm_binary,
                       corpus=corpus,
                       id2word=id2word,
                       time_slices=time_slice,
                       model='dtm',
                       num_topics=8,
                       rng_seed=2020)

2021-01-04 08:32:21,329 : INFO : serializing temporary corpus to /tmp/8d4197_train-mult.dat
2021-01-04 08:32:21,330 : INFO : no word id mapping provided; initializing from corpus
2021-01-04 08:32:21,353 : INFO : storing corpus in Blei's LDA-C format into /tmp/8d4197_train-mult.dat
2021-01-04 08:32:21,414 : INFO : saving vocabulary of 14545 words to /tmp/8d4197_train-mult.dat.vocab
2021-01-04 08:32:21,435 : INFO : training DTM with args --ntopics=8 --model=dtm  --mode=fit --initialize_lda=true --corpus_prefix=/tmp/8d4197_train --outname=/tmp/8d4197_train_out --alpha=0.01 --lda_max_em_iter=10 --lda_sequence_min_iter=6  --lda_sequence_max_iter=20 --top_chain_var=0.005 --rng_seed=2020 
2021-01-04 08:32:21,435 : INFO : Running command ['./dtm_binaries/dtm-linux64', '--ntopics=8', '--model=dtm', '--mode=fit', '--initialize_lda=true', '--corpus_prefix=/tmp/8d4197_train', '--outname=/tmp/8d4197_train_out', '--alpha=0.01', '--lda_max_em_iter=10', '--lda_sequence_min_iter=6', '--lda_sequence_max

In [45]:
# 토픽별 TopN 단어 확인
for i in range(DTM_8.num_topics):
    for t in range(len(DTM_8.time_slices)):
        res = DTM_8.show_topic(i,t,topn=10)
        print('Topic #{}'.format(i), end='\t')
        print(', '.join(w for p, w in res))
    print('\n')

Topic #0	여성, 주체, 욕망, 근대, 근대성, 사랑, 소설, 1960년대, 담론, 몸
Topic #0	여성, 주체, 욕망, 근대, 근대성, 사랑, 소설, 1960년대, 담론, 몸
Topic #0	주체, 여성, 욕망, 근대, 근대성, 사랑, 소설, 1960년대, 담론, 몸
Topic #0	주체, 여성, 근대, 욕망, 근대성, 사랑, 담론, 1960년대, 소설, 몸
Topic #0	주체, 여성, 근대, 욕망, 근대성, 담론, 1960년대, 사랑, 소설, 몸
Topic #0	주체, 여성, 근대, 욕망, 1960년대, 담론, 근대성, 소설, 사랑, 몸
Topic #0	주체, 여성, 근대, 욕망, 1960년대, 담론, 소설, 사랑, 근대성, 몸
Topic #0	주체, 여성, 근대, 욕망, 1960년대, 담론, 소설, 사랑, 몸, 타자
Topic #0	주체, 여성, 근대, 욕망, 1960년대, 소설, 타자, 담론, 사랑, 몸
Topic #0	주체, 여성, 욕망, 근대, 1960년대, 타자, 소설, 사랑, 몸, 담론
Topic #0	주체, 여성, 욕망, 근대, 1960년대, 타자, 소설, 사랑, 몸, 윤리
Topic #0	주체, 여성, 욕망, 근대, 타자, 1960년대, 정치, 사랑, 소설, 윤리
Topic #0	주체, 여성, 욕망, 타자, 근대, 정치, 1960년대, 윤리, 소설, 사랑
Topic #0	주체, 여성, 욕망, 타자, 근대, 정치, 1960년대, 윤리, 소설, 사랑
Topic #0	주체, 여성, 타자, 욕망, 정치, 근대, 1960년대, 소설, 윤리, 몸
Topic #0	주체, 여성, 타자, 정치, 욕망, 근대, 소설, 윤리, 1960년대, 몸
Topic #0	주체, 여성, 타자, 정치, 욕망, 근대, 소설, 윤리, 1960년대, 사랑
Topic #0	주체, 여성, 정치, 타자, 욕망, 소설, 근대, 윤리, 사랑, 공간
Topic #0	주체, 여성, 정치, 타자, 욕망, 소설, 근대, 윤리, 공간, 사랑
Topic #0	주체, 여성, 정치, 욕망, 타

In [48]:
# 모델 저장
!mkdir ./model
DTM_8.save("./model/DTM_8.model")

2021-01-04 08:44:28,086 : INFO : saving DtmModel object under ./model/DTM_8.model, separately None
2021-01-04 08:44:28,330 : INFO : saved ./model/DTM_8.model


### 모델링 결과 시각화